In [ ]:
import os
import PyPDF2
import pandas as pd
from tqdm import tqdm

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    with open(pdf_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text() + "\n\n"
        return text

def process_pdfs(directory):
    """Process all PDFs in the given directory."""
    results = []
    pdf_files = [f for f in os.listdir(directory) if f.lower().endswith('.pdf')]
    
    for pdf_file in tqdm(pdf_files, desc="Processing PDFs"):
        file_path = os.path.join(directory, pdf_file)
        try:
            text = extract_text_from_pdf(file_path)
            results.append({
                'filename': pdf_file,
                'text': text,
                'size': os.path.getsize(file_path)
            })
            print(f"Successfully processed {pdf_file}")
        except Exception as e:
            print(f"Error processing {pdf_file}: {e}")
    
    # Convert to DataFrame
    df = pd.DataFrame(results)
    return df

if __name__ == "__main__":
    data_dir = "data"
    results_df = process_pdfs(data_dir)
    results_df.to_pickle("extracted_pdf_texts.pkl")
    print(f"Processed {len(results_df)} PDF files and saved to extracted_pdf_texts.pkl")

Processing PDFs:   4%|▍         | 1/23 [00:19<07:15, 19.82s/it]

Successfully processed 4000.1hsghhdbk103123.pdf


Processing PDFs:   9%|▊         | 2/23 [00:20<02:55,  8.38s/it]

Successfully processed CCL_BuyersGuide.pdf
Successfully processed renting-vs-owning.pdf


Processing PDFs:  22%|██▏       | 5/23 [00:20<00:41,  2.30s/it]

...

Successfully processed First-TIme-HomeBuyer-Guide.pdf
Successfully processed realestateglossary.pdf


Processing PDFs:  91%|█████████▏| 21/23 [00:26<00:00,  2.81it/s]

Successfully processed home-buyers-guide-1.pdf
Successfully processed renting-vs-buying-study-press-release.pdf


Processing PDFs: 100%|██████████| 23/23 [00:26<00:00,  1.16s/it]

Successfully processed First-TIme-HomeBuyer-Guide-2.pdf
Processed 23 PDF files and saved to extracted_pdf_texts.pkl


In [9]:
import pandas as pd
import random
import os

def check_extracted_data(pkl_path, num_samples=3, sample_length=500):
    """
    Examine the extracted PDF data to check its quality.
    
    Args:
        pkl_path: Path to the pickle file with extracted text
        num_samples: Number of random samples to display
        sample_length: Number of characters to display from each sample
    """
    # Load the data
    if not os.path.exists(pkl_path):
        print(f"Error: {pkl_path} does not exist.")
        return
    
    print(f"Loading data from {pkl_path}...")
    df = pd.read_pickle(pkl_path)
    
    # Print basic information
    print(f"\nDataset contains {len(df)} documents")
    print(f"Columns: {df.columns.tolist()}")
    
    # Check for empty text
    empty_texts = df[df['text'].str.strip() == ''].shape[0]
    print(f"\nDocuments with empty text: {empty_texts}")
    
    # Check text lengths
    df['text_length'] = df['text'].str.len()
    print(f"\nText length statistics:")
    print(df['text_length'].describe())
    
    # Show some random samples
    print(f"\n{num_samples} random samples (first {sample_length} chars):")
    sample_indices = random.sample(range(len(df)), min(num_samples, len(df)))
    
    for i, idx in enumerate(sample_indices):
        doc = df.iloc[idx]
        print(f"\nSample {i+1} from '{doc['filename']}':")
        print("-" * 80)
        print(doc['text'][:sample_length] + "...")
        print("-" * 80)
    
    # Check for common issues
    print("\nChecking for potential issues:")
    
    # Missing spaces between words (possible OCR issue)
    no_spaces = df[~df['text'].str.contains(' ', regex=False)].shape[0]
    print(f"Documents with no spaces (potential OCR issues): {no_spaces}")
    
    # Unusual characters (possible encoding issues)
    unusual_chars = df[df['text'].str.contains('[^\x00-\x7F]', regex=True)].shape[0]
    print(f"Documents with non-ASCII characters: {unusual_chars}")
    
    return df

if __name__ == "__main__":
    check_extracted_data("extracted_pdf_texts.pkl")

Loading data from extracted_pdf_texts.pkl...

Dataset contains 23 documents
Columns: ['filename', 'text', 'size']

Documents with empty text: 0

Text length statistics:
count    2.300000e+01
mean     2.255588e+05
std      8.347313e+05
min      2.083000e+03
25%      2.211300e+04
50%      3.974200e+04
75%      8.795000e+04
max      4.047940e+06
Name: text_length, dtype: float64

3 random samples (first 500 chars):

Sample 1 from 'CCL_BuyersGuide.pdf':
--------------------------------------------------------------------------------
Home 
Buyer’s 
Guide

1

2About Us
Corcoran Classic Living is a top- performing 
residential and commercial real estate firm 
serving greater Athens, GA. Our agents 
are socially-minded and deeply entrenched 
in their communities, backed by a top-
notch support staff, global connections, 
and innovative technologies that ensure a 
seamless experience. Our mission is to treat 
our customers and clients as the lifeblood of 
our business, keep their satisfaction o

In [ ]:
import pandas as pd
import re
import unicodedata
import string
from tqdm import tqdm

def clean_text(text):
    """Clean and preprocess text for NLP tasks."""
    # Replace common non-ASCII characters 
    text = text.replace('–', '-').replace('—', '-').replace(''', "'").replace(''', "'")
    text = text.replace('"', '"').replace('"', '"').replace('…', '...')
    
    # Normalize unicode characters
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    
    # Remove excessive whitespace
    text = re.sub(r'\s+', ' ', text)
    
    # Remove excessive newlines
    text = re.sub(r'\n{3,}', '\n\n', text)
    
    # Clean up page numbers and headers/footers (common in PDFs)
    text = re.sub(r'\n\s*\d+\s*\n', '\n', text)  # Standalone page numbers
    
    # Remove URLs
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
    
    return text.strip()

def chunk_text(text, chunk_size=1000, overlap=100):
    """Split text into overlapping chunks of approximately chunk_size characters."""
    if len(text) <= chunk_size:
        return [text]
    
    chunks = []
    start = 0
    
    # Add timeout protection
    max_iterations = (len(text) // (chunk_size - overlap)) * 2  # Generous upper bound
    iteration = 0
    
    while start < len(text) and iteration < max_iterations:
        iteration += 1
        end = min(start + chunk_size, len(text))
        
        # Limit the search window for breaking points to improve performance
        search_start = max(start, end - 200)
        
        # Try to find a good breaking point (end of sentence or paragraph)
        if end < len(text):
            # Look for paragraph break first (limit search range)
            paragraph_break = text.rfind('\n\n', search_start, end)
            if paragraph_break != -1:
                end = paragraph_break
            else:
                # Look for sentence break (use a simpler, faster approach)
                for marker in ['. ', '! ', '? ']:
                    sentence_break = text.rfind(marker, search_start, end)
                    if sentence_break != -1:
                        end = sentence_break + 2  # +2 to include the punctuation and space
                        break
        
        # Make sure we're making progress
        if end <= start:
            end = start + chunk_size  # Force progress if no break point found
            
        chunks.append(text[start:end].strip())
        start = end - overlap  # Create overlap between chunks
    
    return chunks

def process_text_data(pkl_path, output_path=None, chunk_size=1500):
    """Clean, preprocess and chunk text data from PDFs."""
    # Load the data
    print(f"Loading data from {pkl_path}...")
    df = pd.read_pickle(pkl_path)
    
    # Clean texts
    print("Cleaning text data...")
    df['cleaned_text'] = df['text'].progress_apply(clean_text)
    
    # Chunk texts
    print("Chunking documents into smaller pieces...")
    all_chunks = []
    
    for i, row in tqdm(df.iterrows(), total=len(df), desc="Chunking documents"):
        try:
            # Skip extremely large docs or process them differently
            if len(row['cleaned_text']) > 1_000_000:  # 1 million chars
                print(f"⚠️ Large document detected: {row['filename']} ({len(row['cleaned_text'])} chars)")
                # Process large documents in a simpler way (just divide by size)
                simple_chunks = [row['cleaned_text'][j:j+chunk_size] 
                               for j in range(0, len(row['cleaned_text']), chunk_size)]
                for j, chunk in enumerate(simple_chunks):
                    all_chunks.append({
                        'source_file': row['filename'],
                        'chunk_id': f"{row['filename']}_simple_{j}",
                        'text': chunk.strip()
                    })
                continue
            
            # Regular chunking for normal sized documents
            chunks = chunk_text(row['cleaned_text'], chunk_size=chunk_size)
            for j, chunk in enumerate(chunks):
                all_chunks.append({
                    'source_file': row['filename'],
                    'chunk_id': f"{row['filename']}_{j}",
                    'text': chunk
                })
        except Exception as e:
            print(f"Error processing {row['filename']}: {e}")
    
    chunks_df = pd.DataFrame(all_chunks)
    print(f"Created {len(chunks_df)} chunks from {len(df)} documents")
    
    # Save the processed data
    if output_path:
        chunks_df.to_pickle(output_path)
        print(f"Saved processed chunks to {output_path}")
    
    return chunks_df

if __name__ == "__main__":
    # Add tqdm to pandas operations
    tqdm.pandas()
    
    # Process the data
    processed_df = process_text_data("extracted_pdf_texts.pkl", "processed_chunks.pkl")
    
    # Display some statistics
    print("\nChunk length statistics:")
    processed_df['text_length'] = processed_df['text'].str.len()
    print(processed_df['text_length'].describe())
    
    # Print a few sample chunks
    print("\nSample chunks:")
    for i in range(min(3, len(processed_df))):
        print(f"\nChunk {i+1} from {processed_df.iloc[i]['source_file']}:")
        print("-" * 80)
        print(processed_df.iloc[i]['text'][:300] + "..." if len(processed_df.iloc[i]['text']) > 300 else processed_df.iloc[i]['text'])
        print("-" * 80)

Loading data from extracted_pdf_texts.pkl...
Cleaning text data...


100%|██████████| 23/23 [00:00<00:00, 58.22it/s]


Chunking documents into smaller pieces...


Chunking documents:   0%|          | 0/23 [06:46<?, ?it/s]